In [5]:
%matplotlib inline

from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pylab as plt
from dmba import classificationSummary, gainsChart


In [75]:
#univ.loc[univ['CCAvg']==0,'CC']=0
#univ.loc[univ['CCAvg']!=0,'CC']=1

#univ.CC=univ.CC.astype('int')

## 8.1 개인대부 수용

In [95]:
univ = pd.read_csv('UniversalBank.csv')

In [96]:
univ.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [97]:
univ=univ.rename(columns={'Personal Loan':'Loan','CreditCard':'CC'})

In [98]:
univ.CC=univ.CC.astype('category')
univ.Loan=univ.Loan.astype('category')
univ.Online=univ.Online.astype('category')

In [99]:
univ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   ID                  5000 non-null   int64   
 1   Age                 5000 non-null   int64   
 2   Experience          5000 non-null   int64   
 3   Income              5000 non-null   int64   
 4   ZIP Code            5000 non-null   int64   
 5   Family              5000 non-null   int64   
 6   CCAvg               5000 non-null   float64 
 7   Education           5000 non-null   int64   
 8   Mortgage            5000 non-null   int64   
 9   Loan                5000 non-null   category
 10  Securities Account  5000 non-null   int64   
 11  CD Account          5000 non-null   int64   
 12  Online              5000 non-null   category
 13  CC                  5000 non-null   category
dtypes: category(3), float64(1), int64(10)
memory usage: 444.8 KB


### a

In [100]:
#데이터 분할
train_df, valid_df = train_test_split(univ, test_size=0.4)

In [101]:
train_df

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Loan,Securities Account,CD Account,Online,CC
4982,4983,36,10,45,95126,4,0.2,1,0,0,0,0,0,1
1335,1336,50,24,180,94539,1,1.7,1,0,0,0,0,1,1
392,393,54,29,48,91709,4,1.8,3,0,0,0,0,1,0
3194,3195,41,15,65,90019,3,0.5,3,0,0,0,0,0,0
509,510,52,28,118,90503,2,6.8,1,112,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3081,3082,36,10,78,95064,2,4.5,3,204,0,0,0,1,1
3458,3459,48,23,191,95053,2,2.8,1,231,0,0,0,0,1
4767,4768,35,9,45,90639,3,0.9,1,101,0,1,0,0,0
4042,4043,29,3,190,92612,2,4.5,1,246,0,0,0,1,1


In [102]:
df=train_df[['CC', 'Loan','Online']]

In [103]:
table_1=pd.pivot_table(df, index=['CC', 'Loan'], columns='Online', aggfunc=len)
table_1

Online     0     1
CC Loan           
0  0     766  1146
   1      64   124
1  0     330   484
   1      34    52

In [94]:
# online 열변수, CCAvg 첫번째 행변수, Loan 두번째 행변수
#for predictor in ['CC', 'Loan']:
    # construct the frequency table
 #   df = train_df[['Online', predictor]]
  #  count = df.pivot_table(index='Online', columns=predictor, aggfunc=len)

   # print(count)
    # print() ## 셀 내부의 값을 개수로 표시함

CC         0    1
Online           
0        870  364
1       1217  549

Loan       0    1
Online           
0       1134  100
1       1587  179



### b

In [104]:
## p(loan=1 | cc=1 and online=1)

In [106]:
52/(484+52)

0.09701492537313433

### c

In [108]:
for predictor in ['CC','Online']:
    # construct the frequency table
    df = train_df[['Loan', predictor]]
    freqTable = df.pivot_table(index='Loan', columns=predictor, aggfunc=len)
    
    print(freqTable)
    print()

CC       0    1
Loan           
0     1912  814
1      188   86

Online     0     1
Loan              
0       1096  1630
1         98   176



### d

In [16]:
## 1 ##

In [109]:
86/(188+86)

0.31386861313868614

In [18]:
## 2 ##

In [110]:
176/(1630+176)

0.09745293466223699

In [16]:
## 3 ## && ## 6 ##

In [111]:
print(train_df['Loan'].value_counts() / len(train_df))

0    0.908667
1    0.091333
Name: Loan, dtype: float64


In [23]:
## 4 ##

In [112]:
814/(1912+814)

0.29860601614086574

In [19]:
## 5 ##

In [113]:
1630/(1096+1630)

0.5979457079970653

### e

In [115]:
# 나이브 베이즈 확률 P(Loan = 1 | CC = 1 and Online = 1)

s1=0.3139*0.0975*0.091333
s2=0.2986*0.5979* 0.908667

s1/(s1+s2)

0.016938740830181386

### f

In [116]:
## 완전한 베이즈 확률이 더 정확한 추정이다.

### g

## 8.2 자동차 사고

In [117]:
acc = pd.read_csv('AccidentsFull.csv')
acc

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,SUR_COND,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR
0,0,2,2,1,0,1,0,3,0,0,...,4,0,3,1,1,1,1,0,0,1
1,1,2,1,0,0,1,1,3,2,0,...,4,0,3,2,2,0,0,1,0,0
2,1,2,1,0,0,1,0,3,2,0,...,4,1,2,2,2,0,0,1,0,0
3,1,2,1,1,0,0,0,3,2,0,...,4,1,2,2,1,0,0,1,0,0
4,1,1,1,0,0,1,0,3,2,0,...,4,0,2,3,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42178,0,2,1,0,1,1,0,1,0,0,...,1,2,1,1,1,0,0,1,0,0
42179,1,2,1,1,0,0,0,1,0,0,...,1,0,1,1,1,1,1,0,0,1
42180,0,2,2,0,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,0
42181,1,2,1,1,0,1,0,1,0,0,...,1,0,1,1,1,0,0,1,0,0


In [118]:
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42183 entries, 0 to 42182
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   HOUR_I_R        42183 non-null  int64
 1   ALCHL_I         42183 non-null  int64
 2   ALIGN_I         42183 non-null  int64
 3   STRATUM_R       42183 non-null  int64
 4   WRK_ZONE        42183 non-null  int64
 5   WKDY_I_R        42183 non-null  int64
 6   INT_HWY         42183 non-null  int64
 7   LGTCON_I_R      42183 non-null  int64
 8   MANCOL_I_R      42183 non-null  int64
 9   PED_ACC_R       42183 non-null  int64
 10  RELJCT_I_R      42183 non-null  int64
 11  REL_RWY_R       42183 non-null  int64
 12  PROFIL_I_R      42183 non-null  int64
 13  SPD_LIM         42183 non-null  int64
 14  SUR_COND        42183 non-null  int64
 15  TRAF_CON_R      42183 non-null  int64
 16  TRAF_WAY        42183 non-null  int64
 17  VEH_INVL        42183 non-null  int64
 18  WEATHER_R       42183 non-

In [20]:
acc.MAX_SEV_IR=acc.MAX_SEV_IR.astype('category')
acc.WEATHER_R=acc.WEATHER_R.astype('category')
acc.TRAF_CON_R=acc.TRAF_CON_R.astype('category')

In [21]:
IN=[]

for i in acc['MAX_SEV_IR']:
    if i == 0:
        IN.append('No')
    elif i != 0:
        IN.append('Yes')

In [22]:
acc['INJURY'] = IN
acc.head()

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR,INJURY
0,0,2,2,1,0,1,0,3,0,0,...,0,3,1,1,1,1,0,0,1,Yes
1,1,2,1,0,0,1,1,3,2,0,...,0,3,2,2,0,0,1,0,0,No
2,1,2,1,0,0,1,0,3,2,0,...,1,2,2,2,0,0,1,0,0,No
3,1,2,1,1,0,0,0,3,2,0,...,1,2,2,1,0,0,1,0,0,No
4,1,1,1,0,0,1,0,3,2,0,...,0,2,3,1,0,0,1,0,0,No


### a

In [23]:
len(acc.INJURY=='No')

42183

In [24]:
len(acc.INJURY=='Yes')

42183

### b

In [25]:
acc_b=acc.head(12)

In [26]:
acc_b=acc_b[['INJURY','WEATHER_R','TRAF_CON_R']]
acc_b

,INJURY,WEATHER_R,TRAF_CON_R
0,Yes,1,0
1,No,2,0
2,No,2,1
3,No,1,1
4,No,1,0
5,Yes,2,0
6,No,2,0
7,Yes,1,0
8,No,2,0
9,No,2,0


In [27]:
## 1 ##

table_1=pd.pivot_table(acc_b, index=['WEATHER_R','TRAF_CON_R'], columns='INJURY', aggfunc=len)
table_1

INJURY                 No  Yes
WEATHER_R TRAF_CON_R          
1         0           1.0  2.0
          1           1.0  NaN
          2           1.0  NaN
2         0           5.0  1.0
          1           1.0  NaN

In [28]:
# NaN 결측치로 0으로 생각함
table_1=table_1.fillna(0)
table_1

INJURY                 No  Yes
WEATHER_R TRAF_CON_R          
1         0           1.0  2.0
          1           1.0  0.0
          2           1.0  0.0
2         0           5.0  1.0
          1           1.0  0.0

In [29]:
## 2 ##

In [30]:
# w = 1, t = 0 : 2/3 
# w = 1, t = 1 : 0/1 
# w = 1, t = 2 : 0/1
# w = 2, t = 0 : 1/6
# w = 2, t = 1 : 0/6

In [31]:
## 3 ##

In [32]:
## 4 ##

In [ ]:
## 5 ##

### c

In [33]:
# 데이터 분할
train_df, valid_df = train_test_split(acc, test_size=0.4)

In [34]:
### 1 ###
acc.columns

Index(['HOUR_I_R', 'ALCHL_I', 'ALIGN_I', 'STRATUM_R', 'WRK_ZONE', 'WKDY_I_R',
       'INT_HWY', 'LGTCON_I_R', 'MANCOL_I_R', 'PED_ACC_R', 'RELJCT_I_R',
       'REL_RWY_R', 'PROFIL_I_R', 'SPD_LIM', 'SUR_COND', 'TRAF_CON_R',
       'TRAF_WAY', 'VEH_INVL', 'WEATHER_R', 'INJURY_CRASH', 'NO_INJ_I',
       'PRPTYDMG_CRASH', 'FATALITIES', 'MAX_SEV_IR', 'INJURY'],
      dtype='object')

In [35]:
## 2 ##

In [36]:
acc.head(1)

,HOUR_I_R,ALCHL_I,ALIGN_I,STRATUM_R,WRK_ZONE,WKDY_I_R,INT_HWY,LGTCON_I_R,MANCOL_I_R,PED_ACC_R,...,TRAF_CON_R,TRAF_WAY,VEH_INVL,WEATHER_R,INJURY_CRASH,NO_INJ_I,PRPTYDMG_CRASH,FATALITIES,MAX_SEV_IR,INJURY
0,0,2,2,1,0,1,0,3,0,0,...,0,3,1,1,1,1,0,0,1,Yes


In [37]:
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42183 entries, 0 to 42182
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   HOUR_I_R        42183 non-null  int64   
 1   ALCHL_I         42183 non-null  int64   
 2   ALIGN_I         42183 non-null  int64   
 3   STRATUM_R       42183 non-null  int64   
 4   WRK_ZONE        42183 non-null  int64   
 5   WKDY_I_R        42183 non-null  int64   
 6   INT_HWY         42183 non-null  int64   
 7   LGTCON_I_R      42183 non-null  int64   
 8   MANCOL_I_R      42183 non-null  int64   
 9   PED_ACC_R       42183 non-null  int64   
 10  RELJCT_I_R      42183 non-null  int64   
 11  REL_RWY_R       42183 non-null  int64   
 12  PROFIL_I_R      42183 non-null  int64   
 13  SPD_LIM         42183 non-null  int64   
 14  SUR_COND        42183 non-null  int64   
 15  TRAF_CON_R      42183 non-null  category
 16  TRAF_WAY        42183 non-null  int64   
 17  VEH_INVL    

In [38]:
acc=acc.astype('category')
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42183 entries, 0 to 42182
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   HOUR_I_R        42183 non-null  category
 1   ALCHL_I         42183 non-null  category
 2   ALIGN_I         42183 non-null  category
 3   STRATUM_R       42183 non-null  category
 4   WRK_ZONE        42183 non-null  category
 5   WKDY_I_R        42183 non-null  category
 6   INT_HWY         42183 non-null  category
 7   LGTCON_I_R      42183 non-null  category
 8   MANCOL_I_R      42183 non-null  category
 9   PED_ACC_R       42183 non-null  category
 10  RELJCT_I_R      42183 non-null  category
 11  REL_RWY_R       42183 non-null  category
 12  PROFIL_I_R      42183 non-null  category
 13  SPD_LIM         42183 non-null  category
 14  SUR_COND        42183 non-null  category
 15  TRAF_CON_R      42183 non-null  category
 16  TRAF_WAY        42183 non-null  category
 17  VEH_INVL    

In [39]:
predictors = ['HOUR_I_R', 'ALCHL_I', 'ALIGN_I', 'STRATUM_R', 'WRK_ZONE', 'WKDY_I_R',
       'INT_HWY', 'LGTCON_I_R', 'MANCOL_I_R', 'PED_ACC_R', 'RELJCT_I_R',
       'REL_RWY_R', 'PROFIL_I_R', 'SPD_LIM', 'SUR_COND', 'TRAF_CON_R',
       'TRAF_WAY', 'VEH_INVL', 'WEATHER_R', 'INJURY_CRASH', 'NO_INJ_I',
       'PRPTYDMG_CRASH', 'FATALITIES']
outcome = 'INJURY'

X = pd.get_dummies(acc[predictors])
y = (acc[outcome] == 'Yes').astype(int)
classes = ['No', 'Yes']

# split into training and validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

In [40]:
# run naive Bayes
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)

# predict probabilities
predProb_train = delays_nb.predict_proba(X_train)
predProb_valid = delays_nb.predict_proba(X_valid)

# predict class membership
y_valid_pred = delays_nb.predict(X_valid)
y_train_pred = delays_nb.predict(X_train)

In [41]:
classificationSummary(y_train, y_train_pred, class_names=classes) 

print()


Confusion Matrix (Accuracy 1.0000)

       Prediction
Actual    No   Yes
    No 12392     0
   Yes     0 12917



In [42]:
## 3 ##

In [43]:

classificationSummary(y_valid, y_valid_pred, class_names=classes) 

Confusion Matrix (Accuracy 1.0000)

       Prediction
Actual   No  Yes
    No 8329    0
   Yes    0 8545


In [44]:
## 4 ##

In [45]:
acc=pd.get_dummies(acc, columns = predictors)

In [46]:
df = pd.DataFrame({'actual': 1 - y_valid.cat.codes, 'prob': predProb_valid[:, 0]})
df = df.sort_values(by=['prob'], ascending=False).reset_index(drop=True)

fig, ax = plt.subplots()
fig.set_size_inches(4, 4)
gainsChart(df.actual, ax=ax)

plt.tight_layout()
plt.show()

AttributeError: Can only use .cat accessor with a 'category' dtype

In [47]:
## 5 ##

In [48]:
## 조건부 확률 데이터 검증할때 검증데이터 해야하는지 학습데이터 해야하는지 ??

In [49]:
train_df, valid_df = train_test_split(acc, test_size=0.4)